In [1]:
import sys
import argparse
import logging
import io
import os
import re
import pandas as pd
import math
import random
import numpy as np
from collections import Counter
import json
import requests
import base64 
import Levenshtein as lev
import yaml
import unicodedata
from datetime import datetime, timedelta
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', -1)

/data1/wangxiaonan/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [285]:
import xgboost as xgb
from xgboost import DMatrix
from onnxmltools.convert import convert_xgboost
from onnxmltools.utils import save_model
from onnxmltools.convert.common.data_types import FloatTensorType
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,auc,roc_auc_score
import matplotlib.pyplot as plt

In [142]:
df = pd.read_csv('1.csv', lineterminator='\n',encoding='utf-8-sig', dtype=str)
int_cols = ['']

In [246]:
# process categorical cols, ensure data are in correct range
# subsitute NaN to 999, will be removed after one-hot encoding

In [247]:
test = df.copy()
test = test.apply(preprocessing, axis=1)
test = test[~test['label'].isnull()]
print(len(test))

147486
147484


In [249]:
y = test['label'].tolist()
x = test[['']]

In [ ]:
cate_cols = list(cate_dict.keys())
other_cols = [i for i in list(x) if i not in cate_cols]
enc = OneHotEncoder(categories = [cate_dict[cate_col] + [999] for cate_col in cate_cols])
one_hot_encoded = pd.DataFrame(enc.fit_transform(x[cate_cols]).toarray(), index=x.index,
                              columns=enc.get_feature_names(cate_cols))
one_hot_encoded = one_hot_encoded.drop([cate_col+'_999' for cate_col in cate_cols], axis=1)  # drop na cols
x = pd.concat([x[other_cols], one_hot_encoded], axis=1)

In [253]:
# split to train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

In [271]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
params = {'objective': 'binary:logistic'}
xgb_model = xgb.train(params, dtrain, num_boost_round=100, evals=[(dtrain, 'train'), (dval, 'eval')])

In [301]:
pred = xgb_model.predict(dtest)

In [302]:
predictions = [round(value) for value in pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 75.69%


In [303]:
print ("AUC Score : %f" % roc_auc_score(y_test, predictions))

AUC Score : 0.631178


In [304]:
print(confusion_matrix(y_test,predictions))

[[29215  2584]
 [ 8170  4277]]


In [305]:
print(classification_report(y_test, predictions, labels=[0, 1]))

              precision    recall  f1-score   support

           0       0.78      0.92      0.84     31799
           1       0.62      0.34      0.44     12447

    accuracy                           0.76     44246
   macro avg       0.70      0.63      0.64     44246
weighted avg       0.74      0.76      0.73     44246



In [ ]:
xgb.plot_importance(xgb_model)